In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
from lasagne.utils import floatX

import random
from collections import Counter
import hickle as hkl

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GeForce GTX 960 (CNMeM is disabled)


In [3]:
# prepare corpus
src_corpus = hkl.load('../dataset/beyond.hkl')
print len(src_corpus)


 329
[' CHAPTER I. PREJUDICES OF PHILOSOPHERS'] ['135. Pharisaism is not a deterioration of the good man; a considerable part of it is rather an essential condition of being good.'] ['296. Alas! what are you, after all, my written and painted thoughts! Not long ago you were so variegated, young and malicious, so full of thorns and secret spices, that you made me sneeze and laugh\xe2\x80\x94and now? You have already doffed your novelty, and some of you, I fear, are ready to become truths, so immortal do they look, so pathetically honest, so tedious! And was it ever otherwise? What then do we write and paint, we mandarins with Chinese brush, we immortalisers of things which LEND themselves to writing, what are we alone capable of painting? Alas, only that which is just about to fade and begins to lose its odour! Alas, only exhausted and departing storms and belated yellow sentiments! Alas, only birds strayed and fatigued by flight, which now let themselves be captured with the hand\xe2\x

In [8]:
corpus = ''
for verse in src_corpus:
    corpus = corpus + str(verse[0]) + '\n'

In [4]:
corpus = '\n'.join(src_corpus)
print corpus

TypeError: sequence item 0: expected string, list found

In [79]:
def isPureAscii(msg):
    return all(ord(c) < 128 for c in msg)

In [80]:

en_src_corpus = []
for msg in src_corpus:
    if isPureAscii(msg):
        en_src_corpus.append(msg)

In [83]:
corpus = '\n'.join(en_src_corpus)
#print corpus

In [18]:
VOCABULARY = set(corpus)
VOCAB_SIZE = len(VOCABULARY)
print VOCAB_SIZE

CHAR2IX = {c: i for i,c in enumerate(VOCABULARY)}
IX2CHAR = {i: c for i,c in enumerate(VOCABULARY)}
CHAR_TO_ONEHOT = {c: np.eye(VOCAB_SIZE)[i] for i,c in enumerate(VOCABULARY)}

80


In [19]:
SEQ_LENGTH = 50
BATCH_SIZE = 50
RNN_HIDDEN_SIZE = 200

In [20]:
corpus_train = corpus[:(len(corpus)*9 // 10)]
corpus_valid = corpus[(len(corpus)*9//10):]

In [21]:
# randomly pick sentences 
def data_batch_generator(corpus, size=BATCH_SIZE):
    idx0 = np.random.randint(0,len(corpus) - SEQ_LENGTH -1,size=size)
    
    while True:
        items = np.array([corpus[start:start + SEQ_LENGTH + 1] for start in idx0])
        idx0 = (idx0 + SEQ_LENGTH) % (len(corpus) - SEQ_LENGTH - 1)
        yield items    

In [22]:
# testing batch generator
gen = data_batch_generator(corpus,size=1)
print(next(gen))
print(next(gen))
print(next(gen))

[' and has a lynx-eye for the weak points in those na']
['atures to whose elevations he cannot attain. He is ']
[' confiding, yet only as one who lets himself go, bu']


In [23]:
# After sampling a data batch, we transform it into a one hot feature representation
# and create a target sequence by shifting by one character
def prep_batch_for_network(batch):
    x_seq = np.zeros((len(batch), SEQ_LENGTH, VOCAB_SIZE), dtype='float32')
    y_seq = np.zeros((len(batch), SEQ_LENGTH), dtype='int32')

    for i, item in enumerate(batch):
        for j in range(SEQ_LENGTH):
            x_seq[i, j] = CHAR_TO_ONEHOT[item[j]]
            y_seq[i, j] = CHAR2IX[item[j + 1]]

    return x_seq, y_seq

In [24]:
# Symbolic variables for input. In addition to the usual features and target,
# we need initial values for the RNN layer's hidden states
x_sym = T.tensor3()
y_sym = T.imatrix()
hid_init_sym = T.matrix()
hid2_init_sym = T.matrix()

In [25]:
# Our network has two stacked GRU layers processing the input sequence.
# Before the decoder layer, we need to reshape the sequence into the batch dimension,
# so that timesteps are decoded independently.
l_input = lasagne.layers.InputLayer((None, SEQ_LENGTH, VOCAB_SIZE))

l_rnn = lasagne.layers.GRULayer(l_input,
                                  num_units=RNN_HIDDEN_SIZE,
                                  grad_clipping=5.,
                                  hid_init=hid_init_sym,
                                  )

l_rnn2 = lasagne.layers.GRULayer(l_rnn,
                                  num_units=RNN_HIDDEN_SIZE,
                                  grad_clipping=5.,
                                  hid_init=hid2_init_sym,
                                  )


l_shp = lasagne.layers.ReshapeLayer(l_rnn2, (-1, RNN_HIDDEN_SIZE))

l_decoder = lasagne.layers.DenseLayer(l_shp,
                                      num_units=VOCAB_SIZE,
                                      nonlinearity=lasagne.nonlinearities.softmax)

l_out = lasagne.layers.ReshapeLayer(l_decoder, (-1, SEQ_LENGTH, VOCAB_SIZE))

In [26]:
# We extract the hidden state of each GRU layer as well as the output of the decoder.
# Only the hidden state at the last timestep is needed
hid_out, hid2_out, prob_out = lasagne.layers.get_output([l_rnn, l_rnn2, l_out],
                                                        {l_input: x_sym})

hid_out = hid_out[:, -1]
hid2_out = hid2_out[:, -1]

/usr/local/lib/python2.7/dist-packages/theano/scan_module/scan.py:1019: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)


In [27]:
# We flatten the sequence into the batch dimension before calculating the loss
def calc_cross_ent(net_output, targets):
    preds = T.reshape(net_output, (-1, VOCAB_SIZE))
    targets = T.flatten(targets)
    cost = T.nnet.categorical_crossentropy(preds, targets)
    return cost

loss = T.mean(calc_cross_ent(prob_out, y_sym))

In [28]:
# For stability during training, gradients are clipped and a total gradient norm constraint is also used
MAX_GRAD_NORM = 15

all_params = lasagne.layers.get_all_params(l_out, trainable=True)

all_grads = T.grad(loss, all_params)
all_grads = [T.clip(g, -5, 5) for g in all_grads]
all_grads, norm = lasagne.updates.total_norm_constraint(
    all_grads, MAX_GRAD_NORM, return_norm=True)

updates = lasagne.updates.adam(all_grads, all_params, learning_rate=0.002)

f_train = theano.function([x_sym, y_sym, hid_init_sym, hid2_init_sym],
                          [loss, norm, hid_out, hid2_out],
                          updates=updates
                         )

f_val = theano.function([x_sym, y_sym, hid_init_sym, hid2_init_sym], [loss, hid_out, hid2_out])

In [51]:
# Training takes a while - you may want to skip this and the next cell, and load the pretrained weights instead
hid = np.zeros((BATCH_SIZE, RNN_HIDDEN_SIZE), dtype='float32')
hid2 = np.zeros((BATCH_SIZE, RNN_HIDDEN_SIZE), dtype='float32')

train_batch_gen = data_batch_generator(corpus_train)

for iteration in range(20000):
    x, y = prep_batch_for_network(next(train_batch_gen))
    loss_train, norm, hid, hid2 = f_train(x, y, hid, hid2)
    
    if iteration % 250 == 0:
        print('Iteration {}, loss_train: {}, norm: {}'.format(iteration, loss_train, norm))

ValueError: GpuElemwise. Input dimension mis-match. Input 3 (indices start at 0) has shape[0] == 1, but the output's size on that axis is 50.
Apply node that caused the error: GpuElemwise{Composite{(((i0 - Composite{scalar_sigmoid((i0 + i1))}(i1, i2)) * i3) + (Composite{scalar_sigmoid((i0 + i1))}(i1, i2) * tanh((i4 + (scalar_sigmoid((i5 + i6)) * i7)))))},no_inplace}(CudaNdarrayConstant{[[ 1.]]}, GpuSubtensor{::, int64:int64:}.0, GpuSubtensor{::, int64:int64:}.0, <CudaNdarrayType(float32, matrix)>, GpuSubtensor{::, int64:int64:}.0, GpuSubtensor{::, int64:int64:}.0, GpuSubtensor{::, int64:int64:}.0, GpuSubtensor{::, int64:int64:}.0)
Toposort index: 7
Inputs types: [CudaNdarrayType(float32, (True, True)), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix)]

HINT: Use another linker then the c linker to hav

In [30]:
param_values = lasagne.layers.get_all_param_values(l_out)
d = {'param values': param_values,
     'VOCABULARY': VOCABULARY, 
     'CHAR2IX': CHAR2IX,
     'IX2CHAR': IX2CHAR,
    }
import hickle 
hickle.dump(param_values,'nietzsche_gru_2layer_trained_param_values.pkl')
#pickle.dump(d, open('gru_2layer_trained.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
predict_fn = theano.function([x_sym, hid_init_sym, hid2_init_sym], [prob_out, hid_out, hid2_out])

In [33]:
# Calculate validation loss
hid = np.zeros((BATCH_SIZE, RNN_HIDDEN_SIZE), dtype='float32')
hid2 = np.zeros((BATCH_SIZE, RNN_HIDDEN_SIZE), dtype='float32')

val_batch_gen = data_batch_generator(corpus_valid)

losses = []

for iteration in range(50):
    x, y = prep_batch_for_network(next(val_batch_gen))
    loss_val, hid, hid2 = f_val(x, y, hid, hid2)
    losses.append(loss_val)
print(np.mean(losses))

2.84334


In [34]:
# For faster sampling, we rebuild the network with a sequence length of 1
l_input = lasagne.layers.InputLayer((None, 1, VOCAB_SIZE))

l_rnn = lasagne.layers.GRULayer(l_input,
                                  num_units=RNN_HIDDEN_SIZE,
                                  grad_clipping=5.,
                                  hid_init=hid_init_sym,
                                  )

l_rnn2 = lasagne.layers.GRULayer(l_rnn,
                                  num_units=RNN_HIDDEN_SIZE,
                                  grad_clipping=5.,
                                  hid_init=hid2_init_sym,
                                  )


l_shp = lasagne.layers.ReshapeLayer(l_rnn2, (-1, RNN_HIDDEN_SIZE))

l_decoder = lasagne.layers.DenseLayer(l_shp,
                                      num_units=VOCAB_SIZE,
                                      nonlinearity=lasagne.nonlinearities.softmax)

l_out = lasagne.layers.ReshapeLayer(l_decoder, (-1, 1, VOCAB_SIZE))

hid_out, hid2_out, prob_out = lasagne.layers.get_output([l_rnn, l_rnn2, l_out], {
                        l_input: x_sym,
                    })
hid_out = hid_out[:, -1]
hid2_out = hid2_out[:, -1]
prob_out = prob_out[0, -1]

In [35]:
lasagne.layers.set_all_param_values(l_out, d['param values'])

In [36]:
predict_fn = theano.function([x_sym, hid_init_sym, hid2_init_sym], [prob_out, hid_out, hid2_out])

In [37]:
# We will use random sentences from the validation corpus to 'prime' the network
primers = corpus_valid.split('\n')

In [53]:
# We feed character one at a time from the priming sequence into the network.
# To obtain a sample string, at each timestep we sample from the output probability distribution,
# and feed the chosen character back into the network. We terminate after the first linebreak.
sentence = ''
hid = np.zeros((1, RNN_HIDDEN_SIZE), dtype='float32')
hid2 = np.zeros((1, RNN_HIDDEN_SIZE), dtype='float32')
x = np.zeros((1, 1, VOCAB_SIZE), dtype='float32')

primer = np.random.choice(primers) + '\n'

for c in primer:
    p, hid, hid2 = predict_fn(x, hid, hid2)
    x[0, 0, :] = CHAR_TO_ONEHOT[c]
    
for _ in range(500):
    p, hid, hid2 = predict_fn(x, hid, hid2)
    p = p/(1 + 1e-6)
    s = np.random.multinomial(1, p)
    sentence += IX2CHAR[s.argmax(-1)]
    x[0, 0, :] = s
    if sentence[-1] == '\n':
        break
        
print('PRIMER: ' + primer)
print('GENERATED: ' + sentence)

PRIMER: 293. A man who says: "I like that, I take it for my own, and mean to guard and protect it from every one"; a man who can conduct a case, carry out a resolution, remain true to an opinion, keep hold of a woman, punish and overthrow insolence; a man who has his indignation and his sword, and to whom the weak, the suffering, the oppressed, and even the animals willingly submit and naturally belong; in short, a man who is a MASTER by nature—when such a man has sympathy, well! THAT sympathy has value! But of what account is the sympathy of those who suffer! Or of those even who preach sympathy! There is nowadays, throughout almost the whole of Europe, a sickly irritability and sensitiveness towards pain, and also a repulsive irrestrainableness in complaining, an effeminizing, which, with the aid of religion and philosophical nonsense, seeks to deck itself out as something superior—there is a regular cult of suffering. The UNMANLINESS of that which is called "sympathy" by such groups